# On-the-fly training with ASE
In this notebook, we will see how to run on-the-fly (OTF) training with ASE module

In [32]:
flare_path = "../.." # the path to flare source code folder

import numpy as np
import sys
sys.path.append(flare_path)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Build atomic structure
Let's use bulk Al as a simple example. First of all, we can use ASE to create a cubic unit cell with lattice constant $a=4.05$. Then we build a 1x1x1 super cell for simplicity (it's basically the same as the unit cell, but we can build super cell of other sizes as well). 

(For more examples, see [ASE-Atoms object](https://wiki.fysik.dtu.dk/ase/ase/atoms.html?highlight=atoms#module-ase.atoms), [Using the spacegroup subpackage](https://wiki.fysik.dtu.dk/ase/ase/spacegroup/spacegroup.html))

In [33]:
from ase import Atoms
from ase import units
from ase.spacegroup import crystal
from ase.build import bulk, make_supercell

symbol = 'Al'
a = 4.05  # Angstrom lattice spacing
unit_cell = crystal(symbol, [(0,0,0)], spacegroup=225,
                    cellpar=[a, a, a, 90, 90, 90])
multiplier = np.array([[1,0,0],[0,1,0],[0,0,1]])
super_cell = make_supercell(unit_cell, multiplier)
nat = len(super_cell.positions)
print('number of atoms:', nat)

number of atoms: 4


## Set up gp calculator
Let's import the FLARE module to build a GP force field predictor. 
- We choose `two_plus_three_body` kernel function, meaning our kernel consists of 2-body and 3-body interactions. `two_plus_three_body_grad` is the gradient function of the kernel, which is used in training hyperparameters.
- Hyperparameters are randomly chosen to be: $\sigma_2=0.2$, $l_2=1$, $\sigma_3=0,0001$, $l_3=1$, $noise=0.005$
- The cutoffs of 2-body and 3-body are all set to be $4.5$
- We use 'BFGS' as the algorithm for optimizing hyperparameters
- Then we can create a GP model of class `GaussianProcess`

In [34]:
from flare import kernels
from flare.gp import GaussianProcess

kernel = kernels.two_plus_three_body
kernel_grad = kernels.two_plus_three_body_grad
hyps = np.array([0.2, 1., 0.0001, 1, 0.005])
cutoffs = np.array([4.5, 4.5])
hyp_labels = ['sig2', 'ls2', 'sig3', 'ls3', 'noise']
opt_algorithm = 'BFGS'
gp_model = GaussianProcess(kernel, kernel_grad, hyps, cutoffs,
        hyp_labels, opt_algorithm, par=True)

## Set up mff calculator
If we don't want to use the mapped force field method for GP, we can skip this step. If we want, we need to set up parameters for the mapped force field model.
- `grid_params` is a dict, specifying how the grids are set for the construction of the mapping. `bounds_2` and `bounds_3` are lower & upper bound of the interpolation domain
    - `bounds_2`: lower & upper bound of bond length, the upper bound is usually set to be the 2-body cutoff of GP, and the lower bound is usually set to be 0.05A lower than the minimal interatomic distance
    - `bounds_3`: the three dimensions represent *bond_length_1*, *bond_length_2*, *angle_12*. So the lower bound of the two *bond_length*s are set to be the same as 2-body. And the upper bound is set to be the same as the 3-body cutoff of GP. The angle range is always set to be $[0,\pi]$
- `struc_params` is a dict, specifying parameters of the atomic system
    - `cube_lat` can be set as any cell matrix that is much larger than the super cell we are running with

In [35]:
from flare.mff.mff_new import MappedForceField

grid_params = {'bounds_2': np.array([[3.5], [4.5]]),
               'bounds_3': np.array([[3.5, 3.5, 0.0], [4.5, 4.5, np.pi]]),
               'grid_num_2': 8,
               'grid_num_3': [8, 8, 8],
               'svd_rank_2': 0,
               'svd_rank_3': 0,
               'bodies': '2+3',
               'load_grid': None,
               'load_svd': None,
               'update': False}

struc_params = {'species': 'Al',
                'cube_lat': 100*np.eye(3), #super_cell.cell, 
                'mass_dict': {'Al': 0.000103642695727*27}}

mff_model = MappedForceField(gp_model, grid_params, struc_params)

## Set up FLARE calculator
With GP and MFF ready, we can set up our ASE calculator using `FLARE_Calculator` module. If we want to use MFF method, we need to specify `use_mapping=True`, otherwise its default value is `False`. Then we set the calculator of the super cell as our FLARE calculator.

In [36]:
from flare.modules.ase_calculator import FLARE_Calculator
calc = FLARE_Calculator(gp_model=gp_model, mff_model=mff_model, use_mapping=True)
super_cell.set_calculator(calc)

## Set up DFT calculator
Since the OTF training requires calling DFT for high uncertainty frames, we need to set up the DFT calculator. Note that we can use any calculator that we like, as long as ASE supports it. For a quick presentation, we use [ASE - EAM calculator](https://wiki.fysik.dtu.dk/ase/ase/calculators/eam.html), and we will give examples below on using others.

- Download an EAM potential for Al, here we choose `Al99.eam.alloy`
- create an EAM calculator use ASE module

In [37]:
from ase.calculators.espresso import Espresso
from ase.calculators.eam import EAM

dft_calc = EAM(potential='Al99.eam.alloy')

\* **Below is an example of Quantum Espresso**

In [38]:
# set up input parameters
input_data = {'control':  {'prefix': 'al',
                           'pseudo_dir': './',
                           'outdir': './out',
                           'verbosity': 'high',
                           'calculation': 'scf'},
              'system':   {'ibrav': 0,
                           'ecutwfc': 50,
                           'ecutrho': 300,
                           'smearing': 'mv',
                           'degauss': 0.02,
                           'occupations': 'smearing'},
              'electrons':{'conv_thr': 1.0e-08,
                           #'startingwfc': 'file',
                           'electron_maxstep': 200,
                           'mixing_beta': 0.4}}

dft_input = {'label': 'al',
             'pseudopotential': {'Al', 'Al.pbe-n-kjpaw_psl.1.0.0.UPF'},
             'kpts': (4, 4, 1),
             'input_data': input_data}

# set up scf executable command
pw_loc = "/n/home08/xiey/q-e/bin/pw.x"
no_cpus = 1
npool = 1
pwi_file = dft_input['label'] + '.pwi'
pwo_file = dft_input['label'] + '.pwo'
os.environ['ASE_ESPRESSO_COMMAND'] = 'srun -n {0} --mpi=pmi2 {1} -npool {2} < {3} > {4}'.format(no_cpus, 
                    pw_loc, npool, pwi_file, pwo_file)

# create ASE calculator
dft_calc = Espresso(pseudopotentials=dft_input['pseudopotentials'], label=dft_input['label'], 
                    tstress=True, tprnfor=True, nosym=True, 
                    input_data=dft_input['input_data'], kpts=dft_input['kpts']) 

\* **Below is an example of LAMMPS**

In [ ]:
# set up input parameters
label = 'al'
pot_path = './'
parameters = {'pair_style': 'eam/alloy', 
              'pair_coeff': ['* * Al99.eam.alloy Al'], 
              'mass': ['* 27']}
files = [pot_path+'Al99.eam.alloy']

# set up executable command
os.environ['LAMMPS_COMMAND'] = './lmp_intel_cpu_intelmpi'

# create ASE calculator
lmp_calc = LAMMPS(label=label, keep_tmp_files=True, tmp_dir='./', parameters=parameters, files=files)

## Set up on-the-fly NPT molecular dynamics

In [39]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary, ZeroRotation

# intialize velocity
MaxwellBoltzmannDistribution(super_cell, 200 * units.kB)
Stationary(super_cell)  # zero linear momentum
ZeroRotation(super_cell)  # zero angular momentum

Other OTF thermo-stat modules:

In [40]:
from flare.modules.ase_otf_md import OTF_NPT

timestep = 1 # fs
temperature = 100
externalstress = 0
ttime = 25
pfactor = 3375

test_otf_npt = OTF_NPT(super_cell, timestep, temperature,
                       externalstress, ttime, pfactor, mask=None,
                       # on-the-fly parameters
                       dft_calc=dft_calc,
                       std_tolerance_factor=1, max_atoms_added=nat,
                       freeze_hyps=0,
                       # mff parameters
                       use_mapping=super_cell.calc.use_mapping)

Set up the `OTFLogger` so that we will get a log file, which records all the information of the on-the-fly training process, including the positions, velocities, forces, uncertainties of each step, and the parameters of GP and MFF. The dump interval is set to be $1$, meaning the trajectory is dumped every step.

In [41]:
from flare.modules.ase_otf_logger import OTFLogger
logfile = 'al.log'
test_otf_npt.attach(OTFLogger(test_otf_npt, super_cell, logfile, mode="w"), interval=1)

Now we can run for 5 time steps

In [42]:
test_otf_npt.otf_run(5)

step: 0
calling dft
updating gp
step: 1
calling dft
updating gp
step: 2
calling dft
updating gp
step: 3
calling dft
updating gp
step: 4
calling dft
updating gp


It looks like we are done. Let's look at the first several lines of our log file and see what's in there.

In [43]:
f = open(logfile)
for l in range(20):
    print(f.readline(), end='')

2019-07-31 15:29:05.374015
number of cpu cores: 
cutoffs: [4.5 4.5]
kernel: two_plus_three_body
number of parameters: 5
hyperparameters: [2.e-01 1.e+00 1.e-04 1.e+00 5.e-03]
hyperparameter optimization algorithm: L-BFGS-B
uncertainty tolerance: 1
periodic cell:
[[4.05 0.   0.  ]
 [0.   4.05 0.  ]
 [0.   0.   4.05]]

--------------------------------------------------
-*Frame: 0.0
Simulation time: 0.0
          Positions           |           DFT Forces           |           Velocities          |           Uncertainties
 0.000000  0.000000  0.000000 | -0.000000 -0.000000 -0.000000 |  0.026679 -0.032448  0.010608 |  0.000000  0.000000  0.000000
 0.000000  2.025000  2.025000 |  0.000000 -0.000000  0.000000 | -0.000962  0.027003  0.022478 |  0.000000  0.000000  0.000000
 2.025000  0.000000  2.025000 |  0.000000  0.000000  0.000000 | -0.032124 -0.019127 -0.018484 |  0.000000  0.000000  0.000000
